# Import Event Log

In [45]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe = pd.read_csv('../../data/logs/BPIC12_Log_onlyO.csv', sep=',')

    # Drop the first column without knowing its name
    dataframe = dataframe.drop(dataframe.columns[0], axis=1)

    # Format the dataframe
    dataframe = pm4py.format_dataframe(
        dataframe,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_6931/4192841163.py:14: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe = pm4py.format_dataframe(


# Data Preprocessing

## Integer Encoding 

In [46]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
dataframe['activity_encoded'] = label_encoder.fit_transform(dataframe['concept:name'])

# Now the 'dataframe' has a new column 'activity_encoded' with integer encoded values of the 'activity' column
dataframe

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ,@@index,@@case_index,activity_encoded
0,10862,COMPLETE,O_SELECTED,2011-10-01 09:45:09.243000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,0,0,4
1,10862,COMPLETE,O_CREATED,2011-10-01 09:45:11.197000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,1,0,2
2,10862,COMPLETE,O_SENT,2011-10-01 09:45:11.380000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,2,0,5
3,11049,COMPLETE,O_SENT_BACK,2011-10-10 09:33:03.668000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,3,0,6
4,10629,COMPLETE,O_ACCEPTED,2011-10-13 08:37:29.226000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000,4,0,0
...,...,...,...,...,...,...,...,...,...,...
31239,11003,COMPLETE,O_SENT,2012-03-02 09:14:43.303000+00:00,2012-02-29 22:22:24.570000+00:00,214364,5000,31239,5013,5
31240,10789,COMPLETE,O_SENT_BACK,2012-03-09 10:01:46.008000+00:00,2012-02-29 22:22:24.570000+00:00,214364,5000,31240,5013,6
31241,10933,COMPLETE,O_SELECTED,2012-03-01 19:22:38.593000+00:00,2012-02-29 22:43:09.766000+00:00,214373,8500,31241,5014,4
31242,10933,COMPLETE,O_CREATED,2012-03-01 19:22:40.016000+00:00,2012-02-29 22:43:09.766000+00:00,214373,8500,31242,5014,2


# Build Input X and output Y

In [47]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    
    return windows_padded, targets_array

In [48]:
activity = dataframe[['activity_encoded', '@@case_index']]

In [49]:
X, Y = generate_prefix_windows(activity)

# LSTM Model Architecture

In [50]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Lambda

# Define parameters
input_dim = dataframe['concept:name'].nunique()  # Number of unique activities
embedding_dim = len(dataframe['concept:name'].unique())  # Embedding dimension
lstm_units = 100  # Number of units in the LSTM layer
dropout_rate = 0.2  # Dropout rate to prevent overfitting
output_dim = input_dim  # Output dimension (number of activities)

# Define the input layer
inputs = Input(shape=(None,))

# Embedding layer
embedding = Embedding(input_dim=input_dim, output_dim=embedding_dim)(inputs)

# LSTM layer with return_sequences=True to get hidden states at each time step
lstm_output, lstm_state_h, lstm_state_c = LSTM(units=lstm_units, return_sequences=True, return_state=True)(embedding)

# Apply a Lambda layer to extract the last hidden state
last_hidden_state = Lambda(lambda x: x[:, -1, :])(lstm_output)

# Dropout layer
dropout = Dropout(rate=dropout_rate)(last_hidden_state)

# Dense output layer for next activity prediction
outputs = Dense(units=output_dim, activation='softmax')(dropout)

# Build the model for training (only outputs the prediction)
training_model = Model(inputs=inputs, outputs=outputs)

# Compile the training model
training_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model for inference (outputs both predictions and hidden states)
inference_model = Model(inputs=inputs, outputs=[outputs, lstm_output])

# Display the model architecture
training_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 7)           49        
                                                                 
 lstm_1 (LSTM)               [(None, None, 100),       43200     
                              (None, 100),                       
                              (None, 100)]                       
                                                                 
 lambda_1 (Lambda)           (None, 100)               0         
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 707 

# LSTM Training

In [51]:
from tensorflow.keras.utils import to_categorical

Y = to_categorical(Y, num_classes=input_dim)

In [52]:
history = training_model.fit(X, Y, epochs=25, batch_size=500)

Epoch 1/25
53/53 [==============================] - 15s 187ms/step - loss: 1.8000 - accuracy: 0.2629
Epoch 2/25
53/53 [==============================] - 10s 191ms/step - loss: 1.4325 - accuracy: 0.3635
Epoch 3/25
53/53 [==============================] - 11s 211ms/step - loss: 1.2687 - accuracy: 0.4627
Epoch 4/25
53/53 [==============================] - 11s 216ms/step - loss: 1.0622 - accuracy: 0.5482
Epoch 5/25
53/53 [==============================] - 13s 245ms/step - loss: 1.0050 - accuracy: 0.5635
Epoch 6/25
53/53 [==============================] - 18s 340ms/step - loss: 0.9902 - accuracy: 0.5771
Epoch 7/25
53/53 [==============================] - 16s 295ms/step - loss: 0.9647 - accuracy: 0.6114
Epoch 8/25
53/53 [==============================] - 19s 360ms/step - loss: 0.9114 - accuracy: 0.6547
Epoch 9/25
53/53 [==============================] - 17s 330ms/step - loss: 0.8362 - accuracy: 0.6843
Epoch 10/25
53/53 [==============================] - 19s 358ms/step - loss: 0.7235 - accura

# Extract Trace Representations Using LSTM Network

In [53]:
predictions, hidden_states = inference_model.predict(X)

820/820 [==============================] - 24s 21ms/step


In [54]:
# Group by case index to get individual traces
traces = dataframe.groupby('@@case_index')['concept:name'].apply(list)
traces

@@case_index
0       [O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK, O...
1       [O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...
2       [O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...
3       [O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK, O...
4       [O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...
                              ...                        
5010         [O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK]
5011                      [O_SELECTED, O_CREATED, O_SENT]
5012    [O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK, O...
5013    [O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...
5014                      [O_SELECTED, O_CREATED, O_SENT]
Name: concept:name, Length: 5015, dtype: object

In [55]:
hidden_state_pos = traces.apply(len).values
hidden_state_pos = hidden_state_pos - 1
hidden_state_pos

array([ 4,  8, 12, ...,  4,  7,  2])

In [56]:
trace_representations = [hidden_states[i-1] for i in hidden_state_pos]
trace_representations = np.array(trace_representations)
len(trace_representations)

5015

In [57]:
trace_representations.shape

(5015, 29, 100)

# Ground Truth

In [58]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [59]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_O.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/168 [00:00<?, ?it/s]

In [60]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [61]:
classification = pd.DataFrame(conformance, columns=['actual'])

# Input Traces

In [62]:
# Step 2: Group by case_index and concatenate the activities to form traces
dataframe['trace'] = dataframe.groupby('@@case_index')['concept:name'].transform(lambda x: ', '.join(x))

# Step 3: Count occurrences of each unique trace
trace_counts = dataframe['trace'].value_counts()

# Step 4: Convert to DataFrame and sort by occurrences
trace_counts_df = trace_counts.reset_index()
trace_counts_df.columns = ['Trace', 'Count']
trace_counts_df = trace_counts_df.sort_values(by='Count', ascending=False)

In [63]:
top_3_traces = trace_counts_df.head(20)

trace1 = top_3_traces.iloc[0]['Trace']
trace1 = [event.strip() for event in trace1.split(',')]

trace2 = top_3_traces.iloc[1]['Trace']
trace2 = [event.strip() for event in trace2.split(',')]

trace3 = top_3_traces.iloc[2]['Trace']
trace3 = [event.strip() for event in trace3.split(',')]

trace4 = top_3_traces.iloc[3]['Trace']
trace4 = [event.strip() for event in trace4.split(',')]

trace5 = top_3_traces.iloc[4]['Trace']
trace5 = [event.strip() for event in trace5.split(',')]

trace6 = top_3_traces.iloc[5]['Trace']
trace6 = [event.strip() for event in trace6.split(',')]

trace7 = top_3_traces.iloc[6]['Trace']
trace7 = [event.strip() for event in trace7.split(',')]

trace8 = top_3_traces.iloc[7]['Trace']
trace8 = [event.strip() for event in trace8.split(',')]

trace9 = top_3_traces.iloc[8]['Trace']
trace9 = [event.strip() for event in trace9.split(',')]

trace10 = top_3_traces.iloc[9]['Trace']
trace10 = [event.strip() for event in trace10.split(',')]

trace11 = top_3_traces.iloc[10]['Trace']
trace11 = [event.strip() for event in trace11.split(',')]

trace12 = top_3_traces.iloc[11]['Trace']
trace12 = [event.strip() for event in trace12.split(',')]

trace13 = top_3_traces.iloc[12]['Trace']
trace13 = [event.strip() for event in trace13.split(',')]

trace14 = top_3_traces.iloc[13]['Trace']
trace14 = [event.strip() for event in trace14.split(',')]

trace15 = top_3_traces.iloc[14]['Trace']
trace15 = [event.strip() for event in trace15.split(',')]

In [64]:
grouped = dataframe.groupby('@@case_index')['concept:name'].apply(list).reset_index(name='trace')

In [65]:
def is_happy_trace(row_trace):
    predefined_traces = [trace1, trace13, trace3]
    for trace in predefined_traces:
        if row_trace == trace:
            return 1
    return 0

In [66]:
grouped['happy'] = grouped['trace'].apply(is_happy_trace)
grouped

,@@case_index,trace,happy
0,0,"[O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK, O...",1
1,1,"[O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...",0
2,2,"[O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...",0
3,3,"[O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK, O...",1
4,4,"[O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...",0
...,...,...,...
5010,5010,"[O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK]",0
5011,5011,"[O_SELECTED, O_CREATED, O_SENT]",0
5012,5012,"[O_SELECTED, O_CREATED, O_SENT, O_SENT_BACK, O...",1
5013,5013,"[O_SELECTED, O_CREATED, O_SENT, O_SELECTED, O_...",0


In [67]:
# Get indices of the happy traces in the results dataframe
happy_trace_indices = grouped[grouped['happy'] == 1].index.tolist()

# Extract the corresponding coordinates from the trace_representations array
happy_trace_coordinates = trace_representations[happy_trace_indices]

# Extract unique coordinates
unique_happy_trace_coordinates = np.unique(happy_trace_coordinates, axis=0)

# Assuming the size of unique_happy_trace_coordinates is 3
#happy_trace1, happy_trace2, happy_trace3 = unique_happy_trace_coordinates
happy_trace1 = unique_happy_trace_coordinates

# Distance Measurement

In [68]:
# Flatten each trace representation (7, 100) into a 1D vector of size 700
trace_representations = trace_representations.reshape(trace_representations.shape[0], -1)

# Now flattened_trace_representations will have shape (13087, 700)
happy_trace1 = happy_trace1.flatten()

In [69]:
from scipy.spatial.distance import euclidean

# Calculate the distances to each of the happy traces for every trace representation
distances_to_happy_traces = []

for trace_representation in trace_representations:
    distances = [
        euclidean(trace_representation, happy_trace1)
    ]
    distances_to_happy_traces.append(distances)

# Calculate the average distance to the happy traces for each trace representation
avg_distances = [np.mean(distances) for distances in distances_to_happy_traces]

# Save the distances in a variable
avg_distances_var = np.array(avg_distances)

In [70]:
classification['distance'] = avg_distances_var

# Evaluation

In [71]:
# Determine Threshold

from sklearn.cluster import KMeans

# Filter the DataFrame into conforming and non-conforming subsets
conforming_distances = classification[classification['actual'] == 0]['distance']
non_conforming_distances = classification[classification['actual'] == 1]['distance']

# Determine common bin edges
min_distance = min(classification['distance'])
max_distance = max(classification['distance'])
bin_edges = np.linspace(min_distance, max_distance, num=30)

# Combine the data and reshape for k-means
all_distances = avg_distances_var
all_distances = np.array(all_distances)
all_distances_reshaped = all_distances.reshape(-1, 1)

# Apply k-means clustering
kmeans = KMeans(n_clusters=2, random_state=0).fit(all_distances_reshaped)
kmeans_labels = kmeans.labels_

# Find the threshold as the average of the two cluster centers
threshold_value = np.mean(kmeans.cluster_centers_)

In [72]:
# Assuming 'classification' is a DataFrame and 'threshold_value' is already defined
classification['predicted'] = classification['distance'].apply(lambda x: 1 if x > threshold_value else 0)

In [73]:
# Calculating TP, TN, FP, FN
TP = ((classification['actual'] == 1) & (classification['predicted'] == 1)).sum()
TN = ((classification['actual'] == 0) & (classification['predicted'] == 0)).sum()
FP = ((classification['actual'] == 0) & (classification['predicted'] == 1)).sum()
FN = ((classification['actual'] == 1) & (classification['predicted'] == 0)).sum()

In [74]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 1.00


In [75]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 1.00


In [76]:
precision_no_d = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_d:.2f}")

Precision No Dev: 1.00


In [77]:
# Calculate recall
recall_no_d = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_d:.2f}")

Recall No Dev: 1.00


In [78]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(classification['actual'], classification['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 1.00


# Deviations

### a) identify closest trace

In [79]:
# INPUT TRACE 1



import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/input_traces/bpicO_trace1.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_input_trace1 = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/168 [00:00<?, ?it/s]

In [80]:
# INPUT TRACE 2



import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/input_traces/bpicO_trace13.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_input_trace2 = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/168 [00:00<?, ?it/s]

In [81]:
# INPUT TRACE 3



import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/input_traces/bpic12_trace3.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_input_trace3 = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/168 [00:00<?, ?it/s]

In [82]:
# Calculate Euclidean distance
def euclidean_distance(arr1, arr2):
    return np.linalg.norm(arr1 - arr2)

# Prepare a list to store the results
distance = []

# Iterate through each subarray in one_hot_encoding
for subarray in trace_representations:
    dist_to_trace1 = euclidean_distance(subarray, happy_trace1)
    
    distances = [dist_to_trace1]
    closest_trace_index = np.argmin(distances)
    closest_trace = f'trace_{closest_trace_index + 1}'
    
    distance.append({
        'Distance to Trace 1': dist_to_trace1,
        'Closest Trace': closest_trace
    })

# Create DataFrame
closest_distance = pd.DataFrame(distance)

### b) identify deviations

In [83]:
# Initialize the merged list
aligned_input_trace = []

# Iterate through each row of the fitness dataframe
for index, row in closest_distance.iterrows():
    closest_trace = row['Closest Trace']
    
    # Append the corresponding alignment to the merged list based on the closest trace
    if closest_trace == 'trace_1':
        aligned_input_trace.append(aligned_input_trace1[index])
    elif closest_trace == 'trace_2':
        aligned_input_trace.append(aligned_input_trace2[index])
    elif closest_trace == 'trace_3':
        aligned_input_trace.append(aligned_input_trace3[index])

In [84]:
# Extract fitness values
aligned_traces_fitness = [trace['fitness'] for trace in aligned_traces]
aligned_input_traces_fitness = [trace['fitness'] for trace in aligned_input_trace]

# Create DataFrame
df_fitness = pd.DataFrame({
    'ground_truth_fit': aligned_traces_fitness,
    'predicted_fit': aligned_input_traces_fitness
})

In [85]:
# Find indices where 'predicted' column has value 1
indices_to_keep = classification[classification['predicted'] == 1].index.tolist()

# Filter the lists to keep only the indices where 'predicted' is 1
aligned_input_trace = [aligned_input_trace[i] for i in indices_to_keep]
aligned_traces = [aligned_traces[i] for i in indices_to_keep]

In [86]:
# Function to extract log and model moves excluding (None, >>) and (>>, None)
def extract_moves(alignment):
    log_moves = [move for move in alignment if move[1] == '>>' and move[0] is not None]
    model_moves = [move for move in alignment if move[0] == '>>' and move[1] is not None]
    return log_moves, model_moves

# Initialize counts for moves
total_log_moves = 0
total_no_log_moves = 0
total_model_moves = 0
total_no_model_moves = 0

# Initialize counts for TP, FP, FN, TN
tp_log_moves = 0
fp_log_moves = 0
fn_log_moves = 0
tn_log_moves = 0

tp_model_moves = 0
fp_model_moves = 0
fn_model_moves = 0
tn_model_moves = 0

# Iterate through aligned traces and count moves
for i, aligned_trace in enumerate(aligned_traces):
    log_moves_gt, model_moves_gt = extract_moves(aligned_trace['alignment'])
    total_log_moves += len(log_moves_gt)
    total_no_log_moves += sum(1 for move in aligned_trace['alignment'] if move[1] != '>>' or move[0] is None)
    total_model_moves += len(model_moves_gt)
    total_no_model_moves += sum(1 for move in aligned_trace['alignment'] if move[0] != '>>' or move[1] is None)
    
    if i < len(aligned_input_trace):
        log_moves_input, model_moves_input = extract_moves(aligned_input_trace[i]['alignment'])
        
        # Calculate TP, FP, FN, TN for log moves
        tp_log_moves += sum(1 for move in log_moves_gt if move in log_moves_input)
        fn_log_moves += sum(1 for move in log_moves_gt if move not in log_moves_input)
        fp_log_moves += sum(1 for move in log_moves_input if move not in log_moves_gt)
        tn_log_moves += sum(1 for move in aligned_trace['alignment'] if move not in log_moves_gt and move not in log_moves_input and move[1] != '>>' and move[0] != '>>')
        
        # Calculate TP, FP, FN, TN for model moves
        tp_model_moves += sum(1 for move in model_moves_gt if move in model_moves_input)
        fn_model_moves += sum(1 for move in model_moves_gt if move not in model_moves_input)
        fp_model_moves += sum(1 for move in model_moves_input if move not in model_moves_gt)
        tn_model_moves += sum(1 for move in aligned_trace['alignment'] if move not in model_moves_gt and move not in model_moves_input and move[1] != '>>' and move[0] != '>>')

# Calculate recall, precision, F1 score for log moves
recall_log_moves = tp_log_moves / (tp_log_moves + fn_log_moves) if (tp_log_moves + fn_log_moves) > 0 else 0
precision_log_moves = tp_log_moves / (tp_log_moves + fp_log_moves) if (tp_log_moves + fp_log_moves) > 0 else 0
f1_score_log_moves = 2 * (precision_log_moves * recall_log_moves) / (precision_log_moves + recall_log_moves) if (precision_log_moves + recall_log_moves) > 0 else 0

# Calculate recall, precision, F1 score for model moves
recall_model_moves = tp_model_moves / (tp_model_moves + fn_model_moves) if (tp_model_moves + fn_model_moves) > 0 else 0
precision_model_moves = tp_model_moves / (tp_model_moves + fp_model_moves) if (tp_model_moves + fp_model_moves) > 0 else 0
f1_score_model_moves = 2 * (precision_model_moves * recall_model_moves) / (precision_model_moves + recall_model_moves) if (precision_model_moves + recall_model_moves) > 0 else 0

# Calculate dataset balance for log moves
log_move_percentage = (total_log_moves / (total_log_moves + total_no_log_moves)) * 100 if (total_log_moves + total_no_log_moves) > 0 else 0
no_log_move_percentage = (total_no_log_moves / (total_log_moves + total_no_log_moves)) * 100 if (total_log_moves + total_no_log_moves) > 0 else 0

# Calculate dataset balance for model moves
model_move_percentage = (total_model_moves / (total_model_moves + total_no_model_moves)) * 100 if (total_model_moves + total_no_model_moves) > 0 else 0
no_model_move_percentage = (total_no_model_moves / (total_model_moves + total_no_model_moves)) * 100 if (total_model_moves + total_no_model_moves) > 0 else 0

# Print results for log moves
print(f"Precision (Log Moves): {precision_log_moves:.2f}")
print(f"Recall (Log Moves): {recall_log_moves:.2f}")
print("")

# Print results for model moves
print(f"Precision (Model Moves): {precision_model_moves:.4f}")
print(f"Recall (Model Moves): {recall_model_moves:.4f}")

Precision (Log Moves): 0.83
Recall (Log Moves): 0.96

Precision (Model Moves): 0.4552
Recall (Model Moves): 0.8818


In [87]:
# Filter the other dataframe using the indices_to_keep
df_fitness = df_fitness.loc[indices_to_keep]

In [88]:
from sklearn.metrics import mean_squared_error

# Calculate MSE
mse = mean_squared_error(df_fitness['ground_truth_fit'], df_fitness['predicted_fit'])

# Print the MSE restricted to 4 decimal places
print(f"The Mean Squared Error (MSE) is: {mse:.4f}")

The Mean Squared Error (MSE) is: 0.0003
